In [2]:
import ThreeD_Net_Tools_0203 as D3
import math
import numpy as np #v 1.23.3
import networkx as nx
import pandas as pd
from networkx.algorithms import community
import community
from community import community_louvain #conda install python-louvain
import os
import scipy
import random
import ast
import pickle

In [3]:

def invert_edge_weights (G_list):
    G_list_inverted=[]
    for g in range(len(G_list)):
        G1=G_list[g]
        G=G1.copy(G1)
        for u, v, data in G.edges(data=True):
            data['weight'] = 1/data['weight']
        G_list_inverted.append(G)
    return G_list_inverted


In [37]:
ALL_ANALYSIS='ALL_GAUDI.csv'#finalfilename
directory="/home/ll16598/Documents/GAUDI/Data/publication/"
dir_G=directory+"/analysis_graphs"


In [38]:
TREATS = pd.read_csv(directory+'COLONY_INFO.csv')
analysis_df=TREATS

In [39]:
name_list=[]
for i in TREATS['name']:
    name_list.append(i)

In [40]:
files = [ f.path for f in os.scandir(dir_G)]

In [41]:
all_names=[]
l=0
G_list=[]
while l<len(files) and len(G_list)<100:
    for file in range(0,len(files)):
        day=os.path.basename(files[file])
        filename = "_".join(day.split("_")[:2])  # Split by underscore, take first two parts, join with underscore
        if len(G_list)==100:
            break
        if filename == name_list[l]:
            G=nx.read_graphml(files[file])
            Gs = sorted(nx.connected_components(G), key=len, reverse=True)
            Gmax = G.subgraph(Gs[0])
            attributes = nx.get_node_attributes(Gmax, 'TYPE')
            G_list.append(Gmax)
            all_names.append(filename)
            l+=1
        else:
            l+=0
        if len(G_list)==100:
            break
            l+=1
        else:
            l+=0
G_list_inverted=invert_edge_weights(G_list)

In [42]:
max_edge_diameter_list=[] 
max_edge_spatial_path_list=[]
for G in G_list:
    max_spatial_edges,max_edge_diameter, max_spatial_diameter,max_edge_spatial_path, spatial_edges, spatial_lengths, edge_paths, edge_lengths=D3.calculate_diameter(G)
    max_edge_diameter_list.append(max_edge_diameter)
    max_edge_spatial_path_list.append(max_edge_spatial_path)
analysis_df['unweighted_diameter']=max_edge_diameter_list
analysis_df['weighted_diameter']=max_edge_spatial_path_list

KeyboardInterrupt: 

In [43]:
max_edge_diameter_list

[7, 10, 10, 8, 10, 10, 14, 13]

In [44]:
first_sections = [name.split('_')[0] for name in name_list]
chamber_ids=[]
for g in range(0,len(G_list)):
    G=G_list[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    attributes = nx.get_node_attributes(G, 'TYPE')
    ALL_JUNC_NUMS = [node for node, type_value in attributes.items() if type_value == 'JUNC']
    ALL_CHAM_NUMS = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    nom = first_sections[g]
    ch_ids=[]
    for ch in ALL_CHAM_NUMS:
        ch_ids.append(nom+'_'+ch)
    chamber_ids.append(ch_ids)
analysis_df['chamber_ids'] = chamber_ids

In [48]:

mean_clust_list = []
density_list = []
cham_degree_list = []
mean_chamber_degree_list = []
std_chamber_degree_list = []


cham_degree_centrality_list = []
mean_cham_deg_centrality_list = []
std_cham_deg_centrality_list = []
efficiency=[]

for g in range(0,len(G_list)):
    G=G_list[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    attributes = nx.get_node_attributes(G, 'TYPE')
    ALL_CHAM_NUMS = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    CH1=list(set(ALL_CHAM_NUMS))
    clust_coeff = nx.average_clustering(Gmax)
    clust = nx.clustering(Gmax)
    clust1=list(clust.values())
    clust1=np.array(clust1)
    mean_clust=np.mean(clust1)
    std_clust=np.std(clust1)
    efficiency.append(nx.global_efficiency(G))

    density=nx.density(G)
    degree=nx.degree(G)
    degree = {node: d for node, d in degree}
    cham_degree=[]
    cham_closeness_spatial=[]
    cham_clustering=[]
    for cham in ALL_CHAM_NUMS:
        ch_deg=degree[cham]
        cham_degree.append(ch_deg)

    mean_clust_list.append(mean_clust)
    density_list.append(density)
    cham_degree_list.append(cham_degree)
analysis_df["efficiency"] = efficiency
analysis_df['mean_clustering'] = mean_clust_list
analysis_df['density'] = density_list
analysis_df['chamber_degree'] = cham_degree_list


In [49]:
def information_centrality(G):
    centrality = {}
    # Get the global efficiency of the original graph
    eff = nx.global_efficiency(G)
    for n in G.nodes():
        removed = G.copy()
        removed.remove_node(n)
        new_eff = nx.global_efficiency(removed)
        centrality[n] = (eff - new_eff) / eff
        
    return centrality

In [53]:


modularities=[]
cham_flow_centrality_list=[]
chamber_info_centrality=[] 
idx=2
for g in range(0,len(G_list_inverted)):
    G=G_list_inverted[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    attributes = nx.get_node_attributes(G, 'TYPE')
    ALL_CHAM_NUMS = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    ALL_NE_NUMS = [node for node, type_value in attributes.items() if type_value == 'NEST EN']

    info_cen=information_centrality(G)
    max_centrality = max(info_cen.values())
    info_cen = {node: centrality / max_centrality for node, centrality in info_cen.items()}

    comms = community.best_partition(G)
    comms1=[*set(list(comms.values()))]
    community_num=len(comms1)
    mod=community.modularity(comms,G, weight=None)
    idx+=1
    modularities.append(mod)
    unique_coms = np.unique(list(comms.values()))
    clustering=nx.clustering(G)


    flow_centrality = nx.current_flow_betweenness_centrality(Gmax, weight='weight',\
                                                                normalized=False)

    total_bc = max(flow_centrality.values())
    flow_centrality = {node: centrality/total_bc for node, centrality in flow_centrality.items()}

    cham_flow_cen=[]
    cham_info=[]
    for cham in ALL_CHAM_NUMS:
        
        cham_flow_cen.append(flow_centrality[cham])
        ch_info=info_cen[cham]
        cham_info.append(ch_info)

    cham_flow_centrality_list.append(cham_flow_cen)
    chamber_info_centrality.append(cham_info)


analysis_df["modularity"] = modularities
analysis_df["cham_flow_centrality_list"] = cham_flow_centrality_list
analysis_df['chamber_info_centrality']=chamber_info_centrality


2 0.5280612244897958
3 0.5649430594028931
4 0.4903880384478462
5 0.5135135135135135
6 0.5631944444444444
7 0.6188563327032136
8 0.6618375131888645
9 0.6534604074774207
10 0.6913573407202216
11 0.6878786670918368
12 0.6329639889196675
13 0.6654974489795918
14 0.680908599242834
15 0.6343933906745134
16 0.6830694706994329
17 0.6699032554198151
18 0.696613724196437
19 0.7032437821574179
20 0.7123609533027535
21 0.7328409490571652
22 0.6473469387755103
23 0.6171900826446282
24 0.6230222222222223
25 0.6304240631163708
26 0.6473553719008265
27 0.6754489603024574
28 0.6636284722222223
29 0.6858275917320943
30 0.7024991932303059
31 0.6978625000000002
32 0.5164609053497942
33 0.4589359504132231
34 0.5507692307692307
35 0.5679584775086505
36 0.6370068027210884
37 0.5598958333333333
38 0.6060032449972959
39 0.6399221453287197
40 0.6547555555555556
41 0.5909951325040563
42 0.533203125
43 0.43005671077504726
44 0.4818672839506173
45 0.4856309870887131
46 0.5831760778051966
47 0.4866666666666667
48 0

In [25]:
modularities

[0.5785657110210691,
 0.7498560408536393,
 0.6546817213876286,
 0.6565758819869574,
 0.7338593746012964,
 0.7067020490411347,
 0.7869405333682679,
 0.8032387189177242,
 0.8149995483872404,
 0.8165228038001854,
 0.6930356171252522,
 0.6847646759064802,
 0.7878862032486328,
 0.762725008059931,
 0.6562388513102205,
 0.8058171684150283]

In [ ]:
node_num_list=[]
for G in G_list:
    node_num=len(G.nodes())
    node_num_list.append(node_num)
analysis_df['node_num']=node_num

In [28]:
new_cham_count=[]
new_ne_count=[]
for g in range(0, len(G_list)):
    G=G_list[g]
    
    attributes = nx.get_node_attributes(G, 'TYPE')
    chamber_nodes = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    nest_nodes = [node for node, type_value in attributes.items() if type_value == 'NEST EN']
    new_cham_count.append(len(chamber_nodes))
    new_ne_count.append(len(nest_nodes))
analysis_df['new_cham_count']=new_cham_count
analysis_df['new_ne_count']=new_ne_count


In [ ]:

junction_num=[]
edge_num=[]
for g in G_list:
    edge_num.append(len(g.edges()))
    junction_nodes = [node for node, type_value in attributes.items() if type_value == 'JUNC']
    junction_num.append(len(junction_nodes))
analysis_df['weights']=g_widths
analysis_df['junction_num']=junction_num
analysis_df['edge_num'] = edge_num


In [ ]:
def degree_heterogeneity(G):
    # Get the degrees of all nodes
    degrees = [d for n, d in G.degree()]
    mean_degree = sum(degrees) / len(degrees)
    std_dev_degree = (sum([(d - mean_degree) ** 2 for d in degrees]) / len(degrees)) ** 0.5
    # Compute the coefficient of variation
    cv = std_dev_degree / mean_degree
    return cv

degree_hetero=[]
mean_strength=[]
for g in range(0,len(G_list)):
    G=G_list[g]
    cv=degree_heterogeneity(G)
    degree_hetero.append(cv)
    
analysis_df['degree_hetero']=degree_hetero


In [ ]:
def chamber_connection_proportion(G):
    attributes_OG = nx.get_node_attributes(G, 'TYPE')
    chamber_nodes = [node for node, type_value in attributes_OG.items() if type_value == 'CHAM']
    proportions = {}
    total_chambers = len(chamber_nodes)
    
    for node in chamber_nodes:
        chamber_neighbors = sum(1 for neighbor in G.neighbors(node) if attributes_OG[neighbor] == 'CHAM')
        try:
            proportions[node] = chamber_neighbors / (total_chambers - 1)
        except ZeroDivisionError:
            proportions=np.nan
    
    return proportions

chamber_chamber_degs = []
chamber_chamber_ids=[]
for g in range(0,len(G_list)):
    G=G_list[g]
    chamberids=analysis_df['chamber_ids'][g]
    dd_clean=[]
    cham_ids2=[]
    try:
        dd_vals=list(chamber_connection_proportion(G).values())

        for i in range(0,len(dd_vals)):
            if dd_vals[i] is not np.nan:
                dd_clean.append(dd_vals[i])
                cham_ids2.append(chamberids[i])
    except AttributeError:
        pass
    chamber_chamber_degs.append(dd_clean)
    chamber_chamber_ids.append(cham_ids2)

analysis_df['chamber_chamber_degs']=chamber_chamber_degs
analysis_df['chamber_chamber_ids']=chamber_chamber_ids

In [31]:
analysis_df.to_csv(directory+"/graph_analysis.csv",index=False)
print('SAVED Gs!')

SAVED Gs!
